# Analyzing Startup Fundraising Deals from Crunchbase

## Introduction

In this project, we will reduce memory usage consumed by a dataset. Beyond that, we will load it in chunks into a new SQLite database. Our data is startup investments from Crunchbase.com while working with memory constraints.

Since the information on the startups and their fundraising rounds is always changing and the dataset is from October 2013, the dataset we'll be using isn't completely up to date. It can be downloaded [here](https://github.com/datahoarder/crunchbase-october-2013/blob/master/crunchbase-investments.csv).

The dataset consumes 10.3 megabytes of disk space, and pandas requires 4 to 6 times the amount of space in memory the file does on disk. However, we'll assume we only have 10 megabytes of available memory. 

## Overview of the Data

Since the dataset contains over 50,000 rows, we'll need to read the dataset into dataframes using 5,000 row chunks to ensure that each chunk consumes less than 10 megabytes of memory.

### Missing values

In [1]:
# Calculate missing values in each columns
import pandas as pd
pd.options.display.max_columns = 99
crunchbase_chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

missing = []
for chunk in crunchbase_chunks:
    chunk_null = chunk.isnull().sum()
    missing.append(chunk_null)
    
combined_missing = pd.concat(missing)
combined_missing = combined_missing.groupby(combined_missing.index).sum().sort_values()
print(combined_missing)


company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64


### Memory footprint

In [2]:
# Calculate each column's memory footprint
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

cols_memory = []
for chunk in chunks:
    cols_chunk_memory = chunk.memory_usage(deep=True)
    cols_memory.append(cols_chunk_memory)

combined_cols_memory = pd.concat(cols_memory)
combined_cols_memory = combined_cols_memory.groupby(combined_cols_memory.index).sum().drop('Index')
print(combined_cols_memory)

company_category_code     3421104
company_city              3505926
company_country_code      3172176
company_name              3591326
company_permalink         4057788
company_region            3411585
company_state_code        3106051
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
funding_round_type        3410707
investor_category_code     622424
investor_city             2885083
investor_country_code     2647292
investor_name             3915666
investor_permalink        4980548
investor_region           3396281
investor_state_code       2476607
raised_amount_usd          422960
dtype: int64


In [3]:
# Calculate total memory footprint of all chunks combined in megabytes
total_memory = combined_cols_memory.sum() / (2**20)
total_memory

56.9876070022583

We can drop some columns because they aren't useful for future analysis. For instance, columns with many missing values.

In [4]:
# Read list of the amount of missing in each column and three rows from the dataset
print(combined_missing.sort_values())
three_rows = pd.read_csv('crunchbase-investments.csv', nrows=3)
three_rows

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funding_round_type            3
funded_year                   3
funded_month                  3
funded_at                     3
funded_quarter                3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64


,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000


In [5]:
useless_cols = ['company_permalink', 'investor_permalink', 'investor_category_code', 'funded_at','funded_year', 'funded_month', 'funded_quarter']
useful_cols = chunk.columns.drop(useless_cols).tolist()

useless_cols

['company_permalink',
 'investor_permalink',
 'investor_category_code',
 'funded_at',
 'funded_year',
 'funded_month',
 'funded_quarter']

Reasons we considered these columns useless:
* Columns representing URL's aren't important for our future analysis.
* Column containing too many missing values won't help in our analysis.
* The datetime columns are redundant and aren't important for future questions.

## Selecting Data Types

Before adding the data into SQLite, we can identify the type of each column and see if we can save space by changing types.

In [6]:
# Identify the types for each column
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, encoding='ISO-8859-1')
chunk.dtypes

company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code    float64
investor_country_code     float64
investor_state_code       float64
investor_region            object
investor_city             float64
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year                 int64
raised_amount_usd         float64
dtype: object

However, we don't know if columns shift types across chunks.

In [7]:
# Identify column type in each chunk
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, encoding='ISO-8859-1')
col_types = {}

for chunk in chunks:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk[col].dtypes)]
        else:
            col_types[col].append(str(chunk[col].dtypes))
            
for col in col_types:
    col_types[col] = set(col_types[col])
    
col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'raised_amount_usd': {'float64'}}

These columns that shift the type across chunks can't be represented as numeric types since they are text. That's why we will convert all of them to object type.

In [8]:
# Columns to convert to object type
convert = {'investor_country_code': 'object', 'investor_state_code': 'object', 'investor_city': 'object'}
# Convert these columns to object type
new_col_types = {}
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, dtype=convert, header=0, encoding='ISO-8859-1')

for chunk in chunks:
    for col in chunk.columns:
        if col not in new_col_types:
            new_col_types[col] = [str(chunk[col].dtypes)]
        else:
            new_col_types[col].append(str(chunk[col].dtypes))
            
for col in new_col_types:
    new_col_types[col] = set(new_col_types[col])
    
new_col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'object'},
 'investor_state_code': {'object'},
 'investor_region': {'object'},
 'investor_city': {'object'},
 'funding_round_type': {'object'},
 'raised_amount_usd': {'float64'}}

Now, we can convert some object columns to category type where less than 50% of the values are unique.

In [9]:
# Columns that less than 50% of the values are unique
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, dtype=convert, header=0, encoding='ISO-8859-1')
less_50_cols = []
unique = {}
for chunk in chunks:
    cols = chunk.select_dtypes(include=['object'])
    for col in cols:
        if col not in unique:
            unique[col] = [chunk[col].value_counts()]
        else:
            unique[col].append(chunk[col].value_counts())

combined_unique = {}
for col in cols:
    combined_unique[col] = pd.concat(unique[col])
    combined_unique[col] = combined_unique[col].groupby(combined_unique[col].index).sum()
    per_col = len(combined_unique[col]) / combined_unique[col].sum()
    if per_col < 0.5:
        print(f'{col:21}: {per_col * 100:.2f}% unique values')
        less_50_cols.append(col)

company_name         : 21.89% unique values
company_category_code: 0.08% unique values
company_country_code : 0.00% unique values
company_state_code   : 0.10% unique values
company_region       : 1.03% unique values
company_city         : 2.35% unique values
investor_name        : 19.79% unique values
investor_country_code: 0.18% unique values
investor_state_code  : 0.14% unique values
investor_region      : 1.11% unique values
investor_city        : 2.45% unique values
funding_round_type   : 0.02% unique values


In [10]:
# Convert less_50_cols to category type
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, dtype=convert, header=0, encoding='ISO-8859-1')
new_col_types = {}

for chunk in chunks:
    for col in less_50_cols:
        chunk[col] = chunk[col].astype('category')
        
    for col in chunk.columns:
        if col not in new_col_types:
            new_col_types[col] = [str(chunk[col].dtypes)]
        else:
            new_col_types[col].append(str(chunk[col].dtypes))
            
for col in new_col_types:
    new_col_types[col] = set(new_col_types[col])
    
new_col_types

{'company_name': {'category'},
 'company_category_code': {'category'},
 'company_country_code': {'category'},
 'company_state_code': {'category'},
 'company_region': {'category'},
 'company_city': {'category'},
 'investor_name': {'category'},
 'investor_country_code': {'category'},
 'investor_state_code': {'category'},
 'investor_region': {'category'},
 'investor_city': {'category'},
 'funding_round_type': {'category'},
 'raised_amount_usd': {'float64'}}

We will change the type from the only numeric column to a more efficient subtype since we can't change it to an integer type because of the missing values in this column.

In [11]:
# Convert 'raised_amount_usd' column type to integer
new_col_types = {}
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, dtype=convert, header=0, encoding='ISO-8859-1')

for chunk in chunks:
    for col in less_50_cols:
        chunk[col] = chunk[col].astype('category')
        
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'], downcast='float')
    
    for col in chunk.columns:
        if col not in new_col_types:
            new_col_types[col] = [str(chunk[col].dtypes)]
        else:
            new_col_types[col].append(str(chunk[col].dtypes))
            
for col in new_col_types:
    new_col_types[col] = set(new_col_types[col])
    
new_col_types

{'company_name': {'category'},
 'company_category_code': {'category'},
 'company_country_code': {'category'},
 'company_state_code': {'category'},
 'company_region': {'category'},
 'company_city': {'category'},
 'investor_name': {'category'},
 'investor_country_code': {'category'},
 'investor_state_code': {'category'},
 'investor_region': {'category'},
 'investor_city': {'category'},
 'funding_round_type': {'category'},
 'raised_amount_usd': {'float32'}}

In [12]:
# Calculate new total memory footprint of all chunks combined in megabytes
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, dtype=convert, header=0, encoding='ISO-8859-1')

cols_memory = []
for chunk in chunks:
    for col in less_50_cols:
        chunk[col] = chunk[col].astype('category')
        
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'], downcast='float')
    
    cols_chunk_memory = chunk.memory_usage(deep=True)
    cols_memory.append(cols_chunk_memory)

new_combined_cols_memory = pd.concat(cols_memory)
new_combined_cols_memory = new_combined_cols_memory.groupby(new_combined_cols_memory.index).sum().drop('Index')

new_total_memory = new_combined_cols_memory.sum() / (2**20)
new_total_memory

6.266323089599609

After selecting only useful columns and converting column types, we saved an impressive amount of space since our memory usage went from almost 57 megabytes to approximately 6 megabytes.

# Loading Chunks into SQLite

In [13]:
# Load chunks into a new SQL database file
import sqlite3
chunks = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, dtype=convert, header=0, encoding='ISO-8859-1')
conn = sqlite3.connect('crunchbase.db')

for chunk in chunks:
    for col in less_50_cols:
        chunk[col] = chunk[col].astype('category')
        
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'], downcast='float')
    
    chunk.to_sql('investments', conn, if_exists='append', index=False)

In [14]:
# Overview
pd.read_sql('''SELECT *
                 FROM investments
                LIMIT 5''', conn)

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,raised_amount_usd
0,AdverCar,advertising,USA,CA,SF Bay,San Francisco,1-800-FLOWERS.COM,USA,NY,New York,New York,series-a,2000000.0
1,LaunchGram,news,USA,CA,SF Bay,Mountain View,10Xelerator,USA,OH,Columbus,Columbus,other,20000.0
2,uTaP,messaging,USA,None,United States - Other,None,10Xelerator,USA,OH,Columbus,Columbus,other,20000.0
3,ZoopShop,software,USA,OH,Columbus,columbus,10Xelerator,USA,OH,Columbus,Columbus,angel,20000.0
4,eFuneral,web,USA,OH,Cleveland,Cleveland,10Xelerator,USA,OH,Columbus,Columbus,other,20000.0
